In [16]:
from LLM_pipeline_V2 import LLMCall, ClusteringManager
import base64
from PIL import Image
import io
import os
import numpy as np
import json
import re
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.schema import AIMessage
from langchain_core.runnables import RunnableLambda

In [17]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [18]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [19]:
def prompt_func(data):
    type_ = data["type"]
    text = data["text"]
    content_parts = []

    if type_ == "keywords":
        image = data["image"]
        image_part = {
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{image}",
        }
        content_parts.append(image_part)

    #system_message = SystemMessage(content=data["system_message_text"])

    text_part = {"type": "text", "text": text}

    content_parts.append(text_part)

    human_message = HumanMessage(content=content_parts)

    #return [system_message, human_message]
    return [human_message]


In [20]:
def call_func(chain, prompt):
    try:
        response = chain.invoke(prompt)

        if isinstance(response, AIMessage):
            response_text = response.content
        else:
            response_text = str(response)  # Conversion en string si nécessaire

        print(f"Reponse du llm : {response_text}")

        return response_text

    except Exception as e:
        print(f"Erreur de parsing JSON : {e}. Nouvelle tentative...")
        return -1

In [21]:
directory = "photos_temp"
calling = LLMCall(directory)
df = calling.df
clustering_manager = ClusteringManager(df)
llm = ChatOllama(model="gemma3")

In [22]:
df

,image_name,path,date_time,localisation
0,20240902_154612.jpg,photos_temp\20240902_154612.jpg,2024:09:02 15:46:13,None
1,20240902_154624.jpg,photos_temp\20240902_154624.jpg,2024:09:02 15:46:24,None
2,20240902_155344.jpg,photos_temp\20240902_155344.jpg,2024:09:02 15:53:44,None
3,20240902_161626.jpg,photos_temp\20240902_161626.jpg,2024:09:02 16:16:27,None
4,20240902_162507.jpg,photos_temp\20240902_162507.jpg,2024:09:02 16:25:07,None
5,20240921_131758.jpg,photos_temp\20240921_131758.jpg,2024:09:21 13:17:58,None
6,20240921_131759.jpg,photos_temp\20240921_131759.jpg,2024:09:21 13:17:59,None
7,20240921_135514.jpg,photos_temp\20240921_135514.jpg,2024:09:21 13:55:14,None
8,20240921_135526.jpg,photos_temp\20240921_135526.jpg,2024:09:21 13:55:27,None
9,20240921_135535.jpg,photos_temp\20240921_135535.jpg,2024:09:21 13:55:35,None


In [23]:
cluster_df, clusters_by_day = clustering_manager.perform_neighbors_clustering(threshold=0.6, n_neighbors=3)

Début du clustering par voisins proches...
Génération des embeddings pour le jour: 2024:09:02
Génération des embeddings pour le jour: 2024:09:21
Génération des embeddings pour le jour: 2024:09:28
Génération des embeddings pour le jour: 2024:09:29
Génération des embeddings pour le jour: 2024:10:01
Génération des embeddings pour le jour: 2024:10:22
Génération des embeddings pour le jour: 2024:12:28
Génération des embeddings pour le jour: 2025:01:26
Génération des embeddings pour le jour: 2025:02:13
Génération des embeddings pour le jour: 2025:02:17
Génération des embeddings pour le jour: 2025:02:19
Clustering du jour 2024:09:02 avec 5 images...
Les photos photos_temp\20240902_154612.jpg et photos_temp\20240902_154624.jpg ont une similarité de 0.78
Les photos photos_temp\20240902_154612.jpg et photos_temp\20240902_155344.jpg ont une similarité de 0.50
Les photos photos_temp\20240902_154612.jpg et photos_temp\20240902_161626.jpg ont une similarité de 0.53
Les photos photos_temp\20240902_15

In [24]:
for day, day_clusters in clusters_by_day.items():
    print(f"Day: {day}")
    for cluster_name, image_paths in day_clusters.items():
        print(f"Cluster: {cluster_name}")
        for image_path in image_paths:
            print(f"  {image_path}")

Day: 2024:09:02
Cluster: cluster_1
  photos_temp\20240902_154612.jpg
  photos_temp\20240902_154624.jpg
  photos_temp\20240902_155344.jpg
  photos_temp\20240902_161626.jpg
  photos_temp\20240902_162507.jpg
Day: 2024:09:21
Cluster: cluster_2
  photos_temp\20240921_131758.jpg
  photos_temp\20240921_131759.jpg
  photos_temp\20240921_135514.jpg
  photos_temp\20240921_135535.jpg
  photos_temp\20240921_135526.jpg
Cluster: cluster_3
  photos_temp\20240921_175010.jpg
  photos_temp\20240921_175652.jpg
  photos_temp\20240921_175013.jpg
Cluster: cluster_4
  photos_temp\20240921_175638.jpg
Day: 2024:09:28
Cluster: cluster_5
  photos_temp\20240928_113104.jpg
  photos_temp\20240928_121400.jpg
  photos_temp\20240928_113107.jpg
  photos_temp\20240928_121404.jpg
  photos_temp\20240928_121356.jpg
  photos_temp\20240928_121426.jpg
  photos_temp\20240928_121427.jpg
  photos_temp\20240928_122247.jpg
  photos_temp\20240928_122240.jpg
  photos_temp\20240928_122537.jpg
  photos_temp\20240928_122242.jpg
  photo

In [25]:
def get_representative_image(image_paths):
    embeddings = clustering_manager.image_embedding(paths=image_paths)
    all_scores = []
    for i in range(len(image_paths)):
        similarity = 0
        for j in range(len(image_paths)):
            similarity += np.dot(embeddings[i], embeddings[j])
        all_scores.append(similarity)
    best_score_index = all_scores.index(max(all_scores))
    return image_paths[best_score_index]

In [26]:
def list_best_images(image_paths):
    if len(image_paths) < 4:
        return image_paths
    else:
        embeddings = clustering_manager.image_embedding(paths=image_paths)
        all_scores = []
        for i in range(len(image_paths)):
            similarity = 0
            for j in range(len(image_paths)):
                similarity += np.dot(embeddings[i], embeddings[j])
            all_scores.append(similarity)
        best_score_indices = sorted(range(len(all_scores)), key=lambda i: all_scores[i], reverse=True)[:4]
        return [image_paths[i] for i in best_score_indices]

In [27]:
images_to_send = []
for day, day_clusters in clusters_by_day.items():
    print(f"Day: {day}")
    for cluster_name, image_paths in day_clusters.items():
        if len(image_paths) > 1:
            print(f"Cluster: {cluster_name}")
            representative_images = list_best_images(image_paths)
            print(f"Most representative image: {representative_images}")
            images_to_send.append(representative_images)

Day: 2024:09:02
Cluster: cluster_1
Most representative image: ['photos_temp\\20240902_154612.jpg', 'photos_temp\\20240902_154624.jpg', 'photos_temp\\20240902_162507.jpg', 'photos_temp\\20240902_161626.jpg']
Day: 2024:09:21
Cluster: cluster_2
Most representative image: ['photos_temp\\20240921_135526.jpg', 'photos_temp\\20240921_135535.jpg', 'photos_temp\\20240921_131758.jpg', 'photos_temp\\20240921_135514.jpg']
Cluster: cluster_3
Most representative image: ['photos_temp\\20240921_175010.jpg', 'photos_temp\\20240921_175652.jpg', 'photos_temp\\20240921_175013.jpg']
Day: 2024:09:28
Cluster: cluster_5
Most representative image: ['photos_temp\\20240928_121427.jpg', 'photos_temp\\20240928_122537.jpg', 'photos_temp\\20240928_121426.jpg', 'photos_temp\\20240928_121356.jpg']
Cluster: cluster_6
Most representative image: ['photos_temp\\20240928_132627.jpg', 'photos_temp\\20240928_133600.jpg', 'photos_temp\\20240928_132644.jpg', 'photos_temp\\20240928_132640.jpg']
Day: 2024:09:29
Day: 2024:10:01
D

In [34]:
def llm_call(image_paths, keywords_chain):
    categories = []
    for i in range(0, len(image_paths)):
        print(f"Image {i} : {image_paths[i]}")
        '''image_b64 = encode_image(image_paths[i])
        image_name = os.path.basename(image_paths[i])'''
        images_description = []
        for images in image_paths[i]:
            image_b64 = encode_image(images)
            image_name = os.path.basename(images)
            prompt = {
                "type": "keywords",
                "text": f"write a very short description  of the image",
                "image": image_b64,
            }
            output = call_func(keywords_chain, prompt)
            images_description.append(f"image description : {output}")

        prompt_text = "You'll have descriptions of images stored in the same album :"
        for description in images_description:
            prompt_text += f"\n{description}"
        prompt_text += "\nNow, please summarize the content of these images in a few words suitable for naming a vacation photo album like sports (football, tennis, badminton, bowling, hike) or activity (party, visits, yoga, etc..). Only give the response, no other text."
        print("----")
        print(prompt_text)
        print("----")
        prompt = {
            "type": "keywords",
            "text": prompt_text,
            "image": encode_image(image_paths[i][0])
        }
        output = call_func(keywords_chain, prompt)
        print("============================")
        print(f"RESULT : {output}\n")
        print("===========================")

    return categories

In [32]:
def pipeline_call(image_paths, df, llm):
    image_data = df
    new_image_paths = image_paths

    prompt_chain = RunnableLambda(prompt_func)
    keyword_chain = prompt_chain | llm

    image_data = llm_call(new_image_paths, keyword_chain)

    return image_data

In [35]:
df = pipeline_call(images_to_send, df, llm)

Image 0 : ['photos_temp\\20240902_154612.jpg', 'photos_temp\\20240902_154624.jpg', 'photos_temp\\20240902_162507.jpg', 'photos_temp\\20240902_161626.jpg']
Reponse du llm : Here's a short description of the image:

The image shows a small, rocky waterfall cascading into a clear stream. Lush green trees line the banks, creating a serene and natural scene.
Reponse du llm : Here's a short description of the image:

The image is a distorted, low-angle reflection of a rocky shoreline and a blue sky in a body of water. The reflection is wavy and uneven, creating a dreamlike effect.
Reponse du llm : Here's a short description of the image:

The image shows a paved pathway winding through a lush green forest. A wooden railing runs along one side of the path, leading the eye into the distance. The scene evokes a sense of a peaceful, natural trail.
Reponse du llm : Here's a short description of the image:

Two people are smiling directly at the camera, leaning against each other with their faces 